In [ ]:
# Exercise sheet 7 with the  emotion dataset.csv and seed words.json Load the data set into your console.

# They contain a corpus of Tweets that contain labels for a emotion classification task (anger, joy, sadness and optimism) and a list of possible seed words for each emotion to initialize an LDA with. 

# In a seeded LDA, we force the model to not find topics itself, but the topics we plant into its training using the seed words. 

# Using words referring to certain emotions, we force our LDA to model emotions rather than the content/topics of the Tweets. We will analyze, how we can utilize an unsupervised or a seeded LDA for text classification.

# As this is not a default task for topic modeling, we cannot use the popular implementation but have to refer back to niche implementations, which is common for more niche NLP-tasks. 

# For R, you can use the “seededlda” package, which contains decent documentation in its help-pages. 
# For Python, you will find the file lda model.py in moodle, which is a condensed version of this GitHub repository. See the recommended functions below for more help.

In [1]:
import json
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import confusion_matrix
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


In [2]:
pip install lda

Defaulting to user installation because normal site-packages is not writeable
  Using cached lda-3.0.0.tar.gz (165 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for lda (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [50 lines of output]
      The Meson build system
      Version: 1.7.0
      Source dir: /private/var/folders/cf/grsl9dbx6z9ckwfcv128r6hh0000gq/T/pip-install-7n8f84l2/lda_e1be66f592d54c6da07cbb55f7c5e520
      Build dir: /private/var/folders/cf/grsl9dbx6z9ckwfcv128r6hh0000gq/T/pip-install-7n8f84l2/lda_e1be66f592d54c6da07cbb55f7c5e520/build
      Build type: native build
      Project name: lda
      Project version: undefined
      C compiler for the host machine: cc (clang 15.0.0 "Apple clang version 15.0.0 (clang-1500.3.9.4)")
      C linker for the host machine: cc ld64 1053.12
      Cython compiler fo

In [9]:
# Task 1: load dataset
# option 1

import pandas as pd
import json

# Load the Tweets dataset
tweets_df = pd.read_csv('/Users/oayanwale/Downloads/NLP_Exercise_24_25/Data/emotion_dataset.csv')

# Load the seed words JSON file
with open('/Users/oayanwale/Downloads/NLP_Exercise_24_25/Data/seed_words.json') as f:
    seed_words = json.load(f)

# Display first few rows of tweets and seed words for verification
print(tweets_df.head())
print(seed_words)

                                                text     label
0  “Worry is a down payment on a problem you may ...  optimism
1  My roommate: it's okay that we can't spell bec...     anger
2  No but that's so cute. Atsu was probably shy a...       joy
3  Rooneys fucking untouchable isn't he? Been fuc...     anger
4  it's pretty depressing when u hit pan on ur fa...   sadness
{'anger': ['outrageous', 'infuriating', 'ridiculous', 'absurd', 'exasperating', 'disgusting', 'insulting', 'offensive', 'intolerable', 'unacceptable', 'outrage', 'insane', 'angry', 'upset', 'boiling', 'seething', 'frustrated', 'mad', 'irritated', 'livid', 'indignant', 'agitated', 'annoyed', 'pissed', 'irate', 'aggravated', 'enraged', 'bitter', 'displeased', 'disgruntled', 'vexed', 'temperamental', 'cross', 'testy', 'impatient', 'belligerent', 'furious', 'hostile', 'offended', 'exasperated', 'resentful'], 'sadness': ['sad', 'unhappy', 'melancholy', 'dejected', 'mournful', 'downcast', 'despondent', 'blue', 'dismal', 

In [5]:
# option 2

import pandas as pd
import json

# Define file paths
data_folder = "/Users/oayanwale/Downloads/NLP_Exercise_24_25/Data"
emotion_data_path = f"{data_folder}/emotion_dataset.csv"
seed_words_path = f"{data_folder}/seed_words.json"

# Load dataset
emotion_df = pd.read_csv(emotion_data_path)
print(emotion_df.head())  # Check structure

# Load seed words
with open(seed_words_path, 'r') as f:
    seed_words = json.load(f)
print(seed_words)  # Check structure


                                                text     label
0  “Worry is a down payment on a problem you may ...  optimism
1  My roommate: it's okay that we can't spell bec...     anger
2  No but that's so cute. Atsu was probably shy a...       joy
3  Rooneys fucking untouchable isn't he? Been fuc...     anger
4  it's pretty depressing when u hit pan on ur fa...   sadness
{'anger': ['outrageous', 'infuriating', 'ridiculous', 'absurd', 'exasperating', 'disgusting', 'insulting', 'offensive', 'intolerable', 'unacceptable', 'outrage', 'insane', 'angry', 'upset', 'boiling', 'seething', 'frustrated', 'mad', 'irritated', 'livid', 'indignant', 'agitated', 'annoyed', 'pissed', 'irate', 'aggravated', 'enraged', 'bitter', 'displeased', 'disgruntled', 'vexed', 'temperamental', 'cross', 'testy', 'impatient', 'belligerent', 'furious', 'hostile', 'offended', 'exasperated', 'resentful'], 'sadness': ['sad', 'unhappy', 'melancholy', 'dejected', 'mournful', 'downcast', 'despondent', 'blue', 'dismal', 

# Task 2
# Preprocess the texts so that they are fit for an analysis. Argue the use the preprocessing steps you take for the given analysis.


# Option 1 Task 2: Preprocess Texts
The preprocessing steps might include:

Lowercasing: Convert all text to lowercase.
Removing Punctuation and Numbers: Keep only alphabetic characters.
Tokenization: Split text into individual words.
Stop Word Removal: Remove common words that do not contribute much meaning (like "and", "the", etc.).
Lemmatization/Stemming: Reduce words to their base or root form.

In [3]:
# option 1

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure necessary resources are available
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation and numbers, keeping only alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # Tokenize (split into words)
    words = text.split()
    
    # Remove stopwords and lemmatize remaining words
    processed_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return ' '.join(processed_words)

# Apply preprocessing to tweet texts column (assuming it's named 'text')
tweets_df['processed_text'] = tweets_df['text'].apply(preprocess_text)

# Display processed texts for verification
print(tweets_df[['text', 'processed_text']].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/oayanwale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                                text  \
0  “Worry is a down payment on a problem you may ...   
1  My roommate: it's okay that we can't spell bec...   
2  No but that's so cute. Atsu was probably shy a...   
3  Rooneys fucking untouchable isn't he? Been fuc...   
4  it's pretty depressing when u hit pan on ur fa...   

                                      processed_text  
0  worry payment problem may never joyce meyer mo...  
1  roommate okay spell autocorrect terrible first...  
2     cute atsu probably shy photo cherry helped uwu  
3  rooneys fucking untouchable fucking dreadful d...  
4  pretty depressing u hit pan ur favourite highl...  


# Option 2 Task 2: Preprocessing
Since we're using LDA, we need to preprocess the tweets while preserving sentiment-related words.

Preprocessing Steps:
✅ Lowercasing (keeps words consistent)
✅ Removing punctuation & special characters
✅ Tokenization (splitting words)
✅ Removing stopwords (to keep only meaningful words)
✅ Lemmatization (keeps word meanings but standardizes forms)

In [9]:
# option 2 

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure required downloads
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation and numbers, keeping only alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # Tokenize (split into words)
    words = text.split()
    
    # Remove stopwords and lemmatize remaining words
    processed_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return ' '.join(processed_words)

# Apply preprocessing to the 'text' column of the DataFrame
emotion_df['processed_text'] = emotion_df['text'].apply(preprocess_text)

# Verify that 'processed_text' has been created successfully
print(emotion_df[['text', 'processed_text']].head())

                                                text  \
0  “Worry is a down payment on a problem you may ...   
1  My roommate: it's okay that we can't spell bec...   
2  No but that's so cute. Atsu was probably shy a...   
3  Rooneys fucking untouchable isn't he? Been fuc...   
4  it's pretty depressing when u hit pan on ur fa...   

                                      processed_text  
0  worry payment problem may never joyce meyer mo...  
1  roommate okay spell autocorrect terrible first...  
2     cute atsu probably shy photo cherry helped uwu  
3  rooneys fucking untouchable fucking dreadful d...  
4  pretty depressing u hit pan ur favourite highl...  


[nltk_data] Downloading package punkt to /Users/oayanwale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Task 3
# Train five LDAs with K = 4 topics on your texts. For Python, set n features=10000 and set a high n iter-value. For R, you will need the quanteda::dfm function (see the documentation of seededlda::textmodel lda).

In [8]:
# Load the dataset again (if necessary)
emotion_df = pd.read_csv('/Users/oayanwale/Downloads/NLP_Exercise_24_25/Data/emotion_dataset.csv')

# Print available columns
print("Columns in emotion_df:", emotion_df.columns)

Columns in emotion_df: Index(['text', 'label'], dtype='object')


In [12]:
#option 1

from lda_model import LDA_Model  # Import your custom LDA model

# Convert processed texts into lists of tokenized words.
tokenized_texts = [text.split() for text in tweets_df["processed_text"]]

# Train 5 LDA models with K=4 topics.
lda_models = []
for i in range(5):
    lda_model = LDA_Model(documents=tokenized_texts)  # Pass tokenized texts directly during initialization.
    lda_model.documents_to_topic_model(n_topics=4, n_features=10000, n_iter=200)  # Adjust iterations as needed.
    lda_models.append(lda_model)

# Display topics for the first trained model.
print("Topics from first LDA model:")
lda_models[0].display_topics(n_top_words=5)  # Display top words per topic.

ModuleNotFoundError: No module named 'lda'

# Option 2 Task 3: Train Five LDA Models (Unsupervised)
We train five LDA models with K=4 topics (one for each emotion: anger, joy, sadness, optimism).
We could not use lda_model.py as required. so (instead we use gensim) 

In [6]:
pip install gensim

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
# option 2 Task 3: Train LDA Models
# Use Gensim's LDA Implementation as lda_model was giving error

import pandas as pd
from gensim import corpora, models

# Assuming you've already preprocessed your tweets and stored them in emotion_df['processed_text']
# Convert processed texts into lists of tokenized words.
tokenized_texts = [text.split() for text in emotion_df['processed_text']]

# Create a dictionary representation of the documents.
dictionary = corpora.Dictionary(tokenized_texts)

# Create a bag-of-words corpus from dictionary representation.
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

# Train an LDA model with K=4 topics.
lda_model = models.LdaModel(corpus, num_topics=4, id2word=dictionary, passes=200)

# Display topics found by the model:
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.051*"user" + 0.010*"like" + 0.008*"get" + 0.006*"watch" + 0.005*"feel"')
(1, '0.043*"user" + 0.010*"n" + 0.010*"amp" + 0.008*"like" + 0.008*"people"')
(2, '0.088*"user" + 0.006*"like" + 0.005*"one" + 0.004*"get" + 0.004*"day"')
(3, '0.034*"user" + 0.010*"u" + 0.006*"get" + 0.005*"one" + 0.005*"day"')


# option 3 Task 3: Train LDA Models
lda module is not installed in your Python environment. Instead of using the lda package, we can use a different library such as sklearn.decomposition.LatentDirichletAllocation (LDA from scikit-learn) to train the LDA model.

In [11]:
# option 3

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Convert text into document-term matrix (DTM)
vectorizer = CountVectorizer(max_features=10000)
X = vectorizer.fit_transform(emotion_df['processed_text'])

# Train 5 LDA models with K=4 topics using scikit-learn's LatentDirichletAllocation
n_topics = 4
n_iter = 1000  # Set a high number of iterations for convergence
lda_models = []

for i in range(5):
    lda = LatentDirichletAllocation(n_components=n_topics, max_iter=n_iter, random_state=42)
    lda.fit(X)
    lda_models.append(lda)

# Display top words for each model
for i, lda in enumerate(lda_models):
    print(f"Top words for model {i+1}:")
    for topic_idx, topic in enumerate(lda.components_):
        print(f"  Topic {topic_idx + 1}: ", [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]])


Top words for model 1:
  Topic 1:  ['user', 'like', 'live', 'feel', 'watch', 'day', 'amazing', 'one', 'get', 'love']
  Topic 2:  ['user', 'like', 'people', 'get', 'amp', 'fucking', 'know', 'way', 'sad', 'even']
  Topic 3:  ['user', 'like', 'amp', 'let', 'get', 'today', 'people', 'still', 'one', 'know']
  Topic 4:  ['user', 'go', 'get', 'amp', 'tell', 'someone', 'life', 'want', 'people', 'time']
Top words for model 2:
  Topic 1:  ['user', 'like', 'live', 'feel', 'watch', 'day', 'amazing', 'one', 'get', 'love']
  Topic 2:  ['user', 'like', 'people', 'get', 'amp', 'fucking', 'know', 'way', 'sad', 'even']
  Topic 3:  ['user', 'like', 'amp', 'let', 'get', 'today', 'people', 'still', 'one', 'know']
  Topic 4:  ['user', 'go', 'get', 'amp', 'tell', 'someone', 'life', 'want', 'people', 'time']
Top words for model 3:
  Topic 1:  ['user', 'like', 'live', 'feel', 'watch', 'day', 'amazing', 'one', 'get', 'love']
  Topic 2:  ['user', 'like', 'people', 'get', 'amp', 'fucking', 'know', 'way', 'sad', '

The output indicates that all five LDA models produced very similar topics, which is interesting. Here’s a breakdown of the results and what they might suggest:

### Observations:
1. **Common Terms**: 
   - **User-related Words**: The term "user" appears prominently across multiple topics, suggesting that many tweets may be directed at or about users, possibly in the context of social media interactions.
   - **Emotional Words**: Words like "feel," "love," and "sad" indicate an emotional aspect to the tweets, aligning with the sentiment analysis context.

2. **Repetitive Topics**: 
   - The similarity between topics across different models suggests that the data may have limited diversity in terms of themes or that the model is capturing recurring patterns in language used on Twitter.
   - This repetition can also indicate that the dataset might contain a lot of similar content or expressions.

3. **Potential Overfitting**: If all models yield identical results, it could signal overfitting due to insufficient variation in your training data or parameters not being diverse enough.

### Next Steps
1. **Evaluate Topic Quality**: Consider whether these topics are meaningful representations of distinct themes within your dataset. You might want to examine some actual tweets from each topic for qualitative insights.
   
2. **Adjust Parameters**: You could try experimenting with different values for `n_topics`, `n_iter`, and other hyperparameters to see if you can achieve more varied results.

3. **Visualize Topics**: Consider visualizing these topics using word clouds or bar charts to better understand their composition visually.


In [27]:

#Fix 1: Improve Stopword Removal
#We need to expand stopwords to remove Twitter artifacts and generic words.

ccustom_stop_words = stop_words.union({
    "user", "like", "amp", "n", "u", "one", "get", "im", "back", "day", "really",
    "know", "see", "would", "someone", "time", "life", "good", "need", "make", "watch"
})

def preprocess_text(text):
    text = str(text).lower()  # Lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove punctuation
    words = word_tokenize(text)  # Tokenize
    words = [lemmatizer.lemmatize(word) for word in words if word not in custom_stop_words]  # Remove stop words
    return " ".join(words)

# Apply improved preprocessing
emotion_df["processed_text"] = emotion_df["text"].apply(preprocess_text)


In [28]:
# Fix 2: Increase passes & Use alpha='auto' for Better Topic Separation


from gensim import corpora, models

# Train LDA with improved topic separation
lda_model = models.LdaModel(corpus, num_topics=4, id2word=dictionary, passes=500, alpha='auto')

# Display topics
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)


(0, '0.062*"user" + 0.008*"like" + 0.008*"n" + 0.007*"get" + 0.005*"amp" + 0.005*"people" + 0.005*"u" + 0.004*"think" + 0.004*"even" + 0.003*"feel"')
(1, '0.090*"user" + 0.007*"like" + 0.005*"amp" + 0.005*"go" + 0.005*"one" + 0.004*"want" + 0.004*"back" + 0.004*"u" + 0.004*"watch" + 0.003*"today"')
(2, '0.016*"user" + 0.008*"like" + 0.007*"know" + 0.006*"life" + 0.005*"n" + 0.004*"sad" + 0.004*"feel" + 0.004*"make" + 0.004*"amp" + 0.004*"gbbo"')
(3, '0.024*"user" + 0.007*"get" + 0.006*"n" + 0.005*"time" + 0.005*"day" + 0.005*"im" + 0.004*"fear" + 0.004*"lost" + 0.004*"depression" + 0.004*"anxiety"')


# Task 4
# Train a seeded LDA with the seeds from Task 1 on your texts using the same parameters as in Task 3. Look at the model’s top words. Did the seeding work?


In [20]:
# Load the dataset again (if necessary)
emotion_df = pd.read_csv('/Users/oayanwale/Downloads/NLP_Exercise_24_25/Data/emotion_dataset.csv')

# Print available columns
print("Columns in emotion_df:", emotion_df.columns)

Columns in emotion_df: Index(['text', 'label'], dtype='object')


In [21]:
#preprocess again if necessary 

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure necessary resources are available
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation and numbers, keeping only alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # Tokenize (split into words)
    words = text.split()
    
    # Remove stopwords and lemmatize remaining words
    processed_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return ' '.join(processed_words)

# Apply preprocessing to the 'text' column of the DataFrame
emotion_df['processed_text'] = emotion_df['text'].apply(preprocess_text)

# Verify that 'processed_text' has been created successfully
print(emotion_df[['text', 'processed_text']].head())

                                                text  \
0  “Worry is a down payment on a problem you may ...   
1  My roommate: it's okay that we can't spell bec...   
2  No but that's so cute. Atsu was probably shy a...   
3  Rooneys fucking untouchable isn't he? Been fuc...   
4  it's pretty depressing when u hit pan on ur fa...   

                                      processed_text  
0  worry payment problem may never joyce meyer mo...  
1  roommate okay spell autocorrect terrible first...  
2     cute atsu probably shy photo cherry helped uwu  
3  rooneys fucking untouchable fucking dreadful d...  
4  pretty depressing u hit pan ur favourite highl...  


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/oayanwale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
# option 1

import json
from gensim import corpora, models

# Load the seed words JSON file
with open('/Users/oayanwale/Downloads/NLP_Exercise_24_25/Data/seed_words.json') as f:
    seed_words = json.load(f)

# Prepare seed terms based on emotions from seed_words JSON file.
seed_terms_per_emotion = {emotion: seeds for emotion, seeds in seed_words.items()}

# Convert processed texts into lists of tokenized words (if not done already).
tokenized_texts = [text.split() for text in emotion_df['processed_text']]

# Create a dictionary representation of the documents.
dictionary = corpora.Dictionary(tokenized_texts)

# Create a bag-of-words corpus from dictionary representation.
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

# Train a seeded LDA model (you may need to adapt this part depending on your implementation)
# For example, if using Gensim's built-in methods:
seeded_lda_model = models.LdaModel(corpus, num_topics=4, id2word=dictionary, passes=200)

# Display top terms from seeded model:
print("Topics from seeded LDA model:")
topics_seeded = seeded_lda_model.print_topics(num_words=5)
for topic in topics_seeded:
    print(topic)

Topics from seeded LDA model:
(0, '0.039*"user" + 0.010*"like" + 0.007*"time" + 0.006*"amp" + 0.005*"get"')
(1, '0.120*"user" + 0.007*"amp" + 0.006*"like" + 0.006*"u" + 0.005*"watch"')
(2, '0.010*"get" + 0.005*"anxiety" + 0.005*"one" + 0.005*"someone" + 0.005*"need"')
(3, '0.024*"user" + 0.015*"n" + 0.007*"like" + 0.006*"day" + 0.006*"people"')


# Task 5
# Calculate the most dominant topic for each document for the models from task 3 and 4. 
# Com- pare the results with the true emotion labels for eacht text by using a confusion matrix. 
# Which model does better? 
# Is it good enough or do you think, we need to find a more method more suited to a classifcation task?

In [50]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Function to get dominant topics from an LDA model
def get_dominant_topics(model, corpus):
    """Get dominant topics for each document."""
    dominant_topics = []
    
    for doc_bow in corpus:
        topic_distribution = model.get_document_topics(doc_bow)
        # Find the topic with the highest probability
        dominant_topic_id = max(topic_distribution, key=lambda x: x[1])[0]  
        dominant_topics.append(dominant_topic_id)
        
    return dominant_topics

# Prepare corpus for both models (use bag-of-words representation)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

In [45]:

from sklearn.metrics import confusion_matrix
import numpy as np

# Get dominant topics from regular LDA model (first model)
lda_topic_distribution = lda_models[0].transform(X)  # X is your document-term matrix
tweets_df['dominant_topic'] = np.argmax(lda_topic_distribution, axis=1)  # Get index of max probability topic

# Convert dominant topic predictions to string
tweets_df['dominant_topic'] = tweets_df['dominant_topic'].astype(str)

# Assuming you have a column 'label' representing true emotions,
confusion_mat_lda = confusion_matrix(tweets_df['label'], tweets_df['dominant_topic'])

print("Confusion Matrix for Regular LDA:\n", confusion_mat_lda)


# Now repeat similar steps for seeded LDA model


Confusion Matrix for Regular LDA:
 [[  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [291 375 454 280   0   0   0   0]
 [166 126 266 150   0   0   0   0]
 [ 82  76  75  61   0   0   0   0]
 [187 239 243 186   0   0   0   0]]


In [48]:
# Check data types of relevant columns
print(tweets_df[['label', 'dominant_topic']].dtypes)

# Check data types of relevant columns
print(tweets_df[['label', 'seeded_dominant_topic']].dtypes)

label             object
dominant_topic    object
dtype: object
label                    object
seeded_dominant_topic     int64
dtype: object


In [49]:
# Convert seeded dominant topic predictions to string
#tweets_df['dominant_topic'] = tweets_df['dominant_topic'].astype(str)
tweets_df['seeded_dominant_topic'] = tweets_df['seeded_dominant_topic'].astype(str)

# Now calculate confusion matrices again
confusion_mat_lda = confusion_matrix(tweets_df['label'], tweets_df['dominant_topic'])
print("Confusion Matrix for Regular LDA:\n", confusion_mat_lda)

confusion_mat_seeded_lda = confusion_matrix(tweets_df['label'], tweets_df['seeded_dominant_topic'])
print("Confusion Matrix for Seeded LDA:\n", confusion_mat_seeded_lda)

Confusion Matrix for Regular LDA:
 [[  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [291 375 454 280   0   0   0   0]
 [166 126 266 150   0   0   0   0]
 [ 82  76  75  61   0   0   0   0]
 [187 239 243 186   0   0   0   0]]
Confusion Matrix for Seeded LDA:
 [[  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [326 504 249 321   0   0   0   0]
 [178 287 116 127   0   0   0   0]
 [ 67  94  72  61   0   0   0   0]
 [253 260 143 199   0   0   0   0]]


Observations:
Zero Predictions for Some Classes: The first four rows of both matrices are entirely zeros, indicating that no predictions were made for those categories. This suggests that the model did not identify any documents belonging to these classes.

Class Distribution: In both matrices, certain categories have high counts in specific classes but do not seem to correspond well with the actual distribution of emotions. For example, in the seeded LDA matrix, there are substantial counts for some categories (e.g., 326 predicted as one label).

Overall Performance: The presence of many zeros suggests that both models may be struggling with classification tasks or that certain emotions may not be well represented in your dataset.



In [52]:
# Evaluate performance by comparing matrices
def evaluate_confusion_matrix(confusion_matrix):
    accuracy = np.trace(confusion_matrix) / np.sum(confusion_matrix)
    print(f"Accuracy: {accuracy:.2f}")

print("Evaluation of Regular LDA:")
evaluate_confusion_matrix(confusion_mat_lda)

print("Evaluation of Seeded LDA:")
evaluate_confusion_matrix(confusion_mat_seeded_lda)

Evaluation of Regular LDA:
Accuracy: 0.00
Evaluation of Seeded LDA:
Accuracy: 0.00


In [53]:
# Check some examples where predictions were made
print(tweets_df[['text', 'label', 'dominant_topic', 'seeded_dominant_topic']].head(20))

                                                 text     label  \
0   “Worry is a down payment on a problem you may ...  optimism   
1   My roommate: it's okay that we can't spell bec...     anger   
2   No but that's so cute. Atsu was probably shy a...       joy   
3   Rooneys fucking untouchable isn't he? Been fuc...     anger   
4   it's pretty depressing when u hit pan on ur fa...   sadness   
5   @user but your pussy was weak from what I hear...     anger   
6   Making that yearly transition from excited and...   sadness   
7   Tiller and breezy should do a collab album. Ra...       joy   
8   @user broadband is shocking regretting signing...     anger   
9                 @user Look at those teef! #growl \n     anger   
10  @user @user USA was embarrassing to watch. Whe...     anger   
11  #NewYork: Several #Baloch &amp; Indian activis...   sadness   
12  Your glee filled Normy dry humping of the most...     anger   
13        What a fucking muppet.  @user  #stalker. \n     ange

In [54]:
from sklearn.metrics import confusion_matrix

# Calculate confusion matrix for regular LDA model
confusion_mat_lda = confusion_matrix(tweets_df['label'], tweets_df['dominant_topic'])
print("Confusion Matrix for Regular LDA:\n", confusion_mat_lda)

# Calculate confusion matrix for seeded LDA model
confusion_mat_seeded_lda = confusion_matrix(tweets_df['label'], tweets_df['seeded_dominant_topic'])
print("Confusion Matrix for Seeded LDA:\n", confusion_mat_seeded_lda)

Confusion Matrix for Regular LDA:
 [[  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [291 375 454 280   0   0   0   0]
 [166 126 266 150   0   0   0   0]
 [ 82  76  75  61   0   0   0   0]
 [187 239 243 186   0   0   0   0]]
Confusion Matrix for Seeded LDA:
 [[  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [326 504 249 321   0   0   0   0]
 [178 287 116 127   0   0   0   0]
 [ 67  94  72  61   0   0   0   0]
 [253 260 143 199   0   0   0   0]]


In [55]:
def evaluate_confusion_matrix(confusion_mat):
    """Calculate and print accuracy from confusion matrix."""
    accuracy = np.trace(confusion_mat) / np.sum(confusion_mat)
    print(f"Accuracy: {accuracy:.2f}")

print("Evaluation of Regular LDA:")
evaluate_confusion_matrix(confusion_mat_lda)

print("Evaluation of Seeded LDA:")
evaluate_confusion_matrix(confusion_mat_seeded_lda)

Evaluation of Regular LDA:
Accuracy: 0.00
Evaluation of Seeded LDA:
Accuracy: 0.00


In [56]:
# Check some examples where predictions were made
print(tweets_df[['text', 'label', 'dominant_topic', 'seeded_dominant_topic']].head(20))

                                                 text     label  \
0   “Worry is a down payment on a problem you may ...  optimism   
1   My roommate: it's okay that we can't spell bec...     anger   
2   No but that's so cute. Atsu was probably shy a...       joy   
3   Rooneys fucking untouchable isn't he? Been fuc...     anger   
4   it's pretty depressing when u hit pan on ur fa...   sadness   
5   @user but your pussy was weak from what I hear...     anger   
6   Making that yearly transition from excited and...   sadness   
7   Tiller and breezy should do a collab album. Ra...       joy   
8   @user broadband is shocking regretting signing...     anger   
9                 @user Look at those teef! #growl \n     anger   
10  @user @user USA was embarrassing to watch. Whe...     anger   
11  #NewYork: Several #Baloch &amp; Indian activis...   sadness   
12  Your glee filled Normy dry humping of the most...     anger   
13        What a fucking muppet.  @user  #stalker. \n     ange